<a href="https://colab.research.google.com/github/rsnatorres/analise_dados_ramon/blob/main/2024_07_analise_mercado_imobiliario_rmbh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introdução

In [ ]:
import pandas as pd
import numpy as np
import hashlib
import plotly.express as px
from unidecode import unidecode
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
import seaborn as sns
#from matplotlib_scalebar.scalebar import ScaleBar

In [ ]:
path_imagens = r'D:\OneDrive\Academia\0 - Artigos\2024 - Capital Imobiliário - RMBH\fwddadoszpbh\graficos_ramon'
#path_imagens = r'C:\Users\Lord-\OneDrive\Academia\0 - Artigos\2024 - Capital Imobiliário - RMBH\fwddadoszpbh\graficos_ramon'

# Base de Dados

In [ ]:
df_bh = pd.read_excel('BH.xlsx', header = 2)
#df_bh['cidade'] = 'Belo Horizonte'
df_betim = pd.read_excel('Betim.xlsx', header = 2)
#df_betim['cidade'] = 'Betim'
df_contagem = pd.read_excel('Contagem.xlsx', header = 2)
#df_contagem['cidade'] = 'Contagem'
df_lagoa_santa = pd.read_excel('Lagoa Santa.xlsx', header = 2)
#df_lagoa_santa['cidade'] = 'Lagoa Santa'
df_nova_lima = pd.read_excel('Nova Lima.xlsx', header = 2)
#df_nova_lima['cidade'] = 'Nova Lima'
df = pd.concat([df_bh, df_betim, df_lagoa_santa, df_nova_lima, df_contagem])
df.reset_index(inplace=True)
df.drop('index', axis=1, inplace=True)

In [ ]:
ipca = pd.read_csv('ipca_mensal_bacen.csv',
               sep=';', decimal=',', header=0, encoding='latin1')
ipca.drop(ipca.tail(1).index, inplace=True) # dropando a fonte
ipca.rename(columns={'Data': 'data',
        '433 - Índice nacional de preços ao consumidor-amplo (IPCA) - Var. % mensal': 'ipca'},
    inplace=True)
ipca['data'] = pd.to_datetime(ipca['data'], format = '%m/%Y')
ipca['ipca'] = pd.to_numeric(ipca['ipca'].str.replace(',','.'))/100
# DEFINIR A DATA BASE --------------------------------------------------------
ipca = ipca[ipca['data'] < '2024']
# chave para depois mergiar
ipca['mes/ano'] = ipca['data'].dt.strftime('%m/%Y')
# preparando para utilizar cumulative product
ipca['ipca_1'] = ipca['ipca'] + 1
# cumulative product
ipca['ipca_cumprod'] = ipca['ipca_1'].cumprod().round(4)
# building the index --> para valores a preços presentes (ultima obs)
ipca['multiplier'] = (ipca['ipca_cumprod'].iloc[-1]/ipca['ipca_cumprod']).round(2)

## Variáveis:

Identificadores:
- rgi: parece ser um registro/identificador do empreendimento
- Código no Mapa: também, mas para relacionar com o .kml
- Empreendimento: Nome para identificar o empreendimento

Essas três variáveis compõe as linhas, que parecem compor tipologias relacionadas a cada empreendimento, essa parece ser a unidade básica de observação do banco de dados

In [ ]:
df.head(2)

- Agregar comercial residencial, mas filtrar análise posterior por residencial
- separar loteamento (fazer mapa próprio)

## Transformações

### Schema

In [ ]:
colunas_data = ['Data Lançamento', 'Data Entrega', 'Data Ocupação',
                'Data Projeto Aprovado', 'Data Pesquisa']
for coluna in colunas_data:
    df[coluna] = pd.to_datetime(df[coluna])
df['ano_entrega'] = df['Data Entrega'].dt.year

In [ ]:
df['mes/ano'] = df['Data Entrega'].dt.strftime('%m/%Y')

In [ ]:
# possível identificador nos dados
colunas_identificadoras = ['RGI', 'Código no Mapa', 'Empreendimento', 'Unidade']
for coluna in colunas_identificadoras:
    df[coluna] = df[coluna].astype(str)

In [ ]:
def normalize_text(text):
    return unidecode(text)

In [ ]:
df.columns = df.columns.str.lower().str.replace(
    ' ', '_').str.replace(
    '%', 'perct').str.replace(
    '(', '').str.replace(
    ')', '_').str.replace(
    'º', '')

In [ ]:
df.columns = [normalize_text(col) for col in df.columns]

### Residencial X Comercial

In [ ]:
df.pivot_table(index = ['tipologia', 'unidade'],
               values = 'n_total_de_unidades',
               aggfunc = 'sum')

In [ ]:
conditions = [df['tipologia'] == 'Comercial',
              df['tipologia'] == 'Comercial - Empresa',
              df['tipologia'] == 'Comercial - Office',
              df['tipologia'] == 'Loteamento',
              df['tipologia'] == 'Res. Vertical',
              df['tipologia'] == 'Res. Horizontal',
              df['tipologia'] == 'Flat/ Hotel',
              df['tipologia'] == 'Retrofit Residencial',
              df['tipologia'] == 'Retrofit Flat/Hotel',
              df['tipologia'] == 'Outros Estados']
values = ['Comercial',
          'Comercial',
          'Comercial',
          'Loteamento',
          'Residencial',
          'Residencial',
          'Residencial',
          'Residencial',
          'Residencial',
          'Residencial']
df['finalidade'] = np.select(conditions, values)

### Deflacionamento

In [ ]:
df = df.merge(ipca[['mes/ano', 'multiplier']], on='mes/ano', how='left')
df['vmu_preco_de_venda_da_unidade_no_lancamento_def'] = (
    df['vmu_preco_de_venda_da_unidade_no_lancamento']*df['multiplier']
                                          ).round(2)
df['vgv_def'] = (
    df['vmu_preco_de_venda_da_unidade_no_lancamento']*df['multiplier']
                                          ).round(2)

In [ ]:
df['finalidade'].value_counts(normalize=True
)

## unidades (tipos)

In [ ]:
df.loc[df['unidade'] == 'COB DUPLEX', 'unidade'] = 'COBERTURA'
df.loc[df['unidade'] == 'COB TRIPLEX', 'unidade'] = 'COBERTURA'
df.loc[df['unidade'] == 'PENTHOUSE', 'unidade'] = 'COBERTURA'
df.loc[df['unidade'] == 'LOFT DUPLEX', 'unidade'] = 'DUPLEX'
df.loc[df['unidade'] == 'SUPERIOR', 'unidade'] = 'OUTROS'
df.loc[df['unidade'] == 'TRIPLEX', 'unidade'] = 'OUTROS'
df.loc[df['unidade'] == '5 DORMS', 'unidade'] = 'OUTROS'
df.loc[df['unidade'] == 'MAISON', 'unidade'] = 'OUTROS'
df.loc[df['unidade'] == 'TERRACE', 'unidade'] = 'OUTROS'
df.loc[df['unidade'] == 'nan', 'unidade'] = 'OUTROS'
df.loc[df['unidade'] == 'SOBRADO', 'unidade'] = 'OUTROS'
df.loc[df['unidade'] == 'STUDIO', 'unidade'] = 'OUTROS'

### Hash

In [ ]:
len(df), df['rgi'].nunique(), df['codigo_no_mapa'].nunique() # rgi é registro do empreendimento?
# se estivermos corretos, então nem todos os rgis foram espacializados no Mapa

In [ ]:
colunas_unicas_p_cada_linha = ['rgi', 'unidade', 'area_privativa',
                               'vmu_preco_de_venda_da_unidade_no_lancamento']

In [ ]:
df[colunas_unicas_p_cada_linha].head(2)

In [ ]:
df['raw_hash'] = (df['rgi'].astype(str)  +
                  df['unidade'].astype(str)
                  + df['area_privativa'].astype(str)
                  + df['vmu_preco_de_venda_da_unidade_no_lancamento'].astype(str) )

In [ ]:
def create_hash(value):
    hash_object = hashlib.sha256()  # a better algorithm, also demands more mem
    hash_object.update(value.encode('utf-8'))
    return hash_object.hexdigest()

In [ ]:
# creating the hash
df['hash'] = df['raw_hash'].apply(create_hash)

In [ ]:
df['hash'].nunique(), len(df) # ok, sem hash collisions

## Filtro

In [ ]:
df.head(2)

In [ ]:
df = df[['rgi', 'codigo_no_mapa', 'empreendimento', 'endereco', 'bairro',
           'cidade', 'data_lancamento', 'data_entrega', 'mes/ano','classificacao_do_tipo',
           'incorporadora_1', 'construtora_1', 'vendedora_1', 'banco', 'unidade',
           'area_privativa', 'n_total_de_unidades', 'n_de_pavimentos',
           'vmu_preco_de_venda_da_unidade_no_lancamento', 'vmu_preco_de_venda_da_unidade_no_lancamento_def',
           'vuv_preco_m2_privativo_no_lancamento', 'vgv', 'vgv_def', 'hash', 'ano_entrega',
           'finalidade', 'tipologia']]

In [ ]:
df['ano_entrega'].value_counts()

In [ ]:
len(df)

In [ ]:
df = df.query("ano_entrega >= 2010 & ano_entrega <= 2023")

In [ ]:
len(df)

## Checks

In [ ]:
df['vgv_check'] = df['n_total_de_unidades'] * df['vmu_preco_de_venda_da_unidade_no_lancamento']

In [ ]:
(df['vgv_check'] - df['vgv']).value_counts()

In [ ]:
#df.to_csv('base_pronta.csv', index=False)

# Análise dos dados

In [ ]:
df.head(2)

In [ ]:
len(df)

## Finalidade

In [ ]:
tab_finalidade_ano = df.pivot_table(index=['ano_entrega', 'finalidade'],
                                    values = 'n_total_de_unidades',
                                    aggfunc = 'sum')
tab_finalidade_ano.reset_index(inplace=True)
tab_finalidade_ano.head(2)

In [ ]:
tab_finalidade = tab_finalidade_ano.pivot_table(index='finalidade',
                                           values = 'n_total_de_unidades',
                                           aggfunc = 'sum').reset_index()
tab_finalidade

In [ ]:
tab_finalidade['proporcao'] = tab_finalidade['n_total_de_unidades'].apply(
    lambda x: x/tab_finalidade['n_total_de_unidades'].sum())

In [ ]:
tab_finalidade

In [ ]:
tab_finalidade['n_total_de_unidades'].sum()

In [ ]:
fig = px.bar(tab_finalidade_ano,
               x = 'ano_entrega',
               y = 'n_total_de_unidades',
               color = 'finalidade',
               barmode = 'group')
# Update x-axis to show all tick labels
fig.update_xaxes(tickangle=-45, tickmode='linear')

In [ ]:
#fig.write_image(fr"{path_imagens}\finalidade_por_ano.png")

In [ ]:
tab_unidades = df.pivot_table(index='finalidade',
                values = 'n_total_de_unidades',
                aggfunc = 'sum')
tab_unidades.reset_index(inplace=True)
tab_unidades['perct_unidades'] = round(tab_unidades['n_total_de_unidades']/tab_unidades['n_total_de_unidades'].sum(), 4)

In [ ]:
tab_unidades

Empreendimentos

In [ ]:
df.head(3)

In [ ]:
df['rgi'].nunique()

In [ ]:
df2 = df[['rgi', 'finalidade']].copy()

In [ ]:
df2 = df2.drop_duplicates(subset='rgi')

In [ ]:
tab_finalidade_empreendimento = df2.pivot_table(index='finalidade',
                                               values = 'rgi',
                                               aggfunc='count').reset_index()
tab_finalidade_empreendimento

In [ ]:
tab_finalidade_empreendimento['proporcao'] = tab_finalidade_empreendimento['rgi'].apply(
    lambda x: x/tab_finalidade_empreendimento['rgi'].sum())

In [ ]:
tab_finalidade_empreendimento

In [ ]:
tab_finalidade_empreendimento['rgi'].sum()

# MatplotLib Style

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates # Para alterar datas no eixo
from matplotlib.dates import DateFormatter # Para alterar o formato da data
from matplotlib.ticker import FuncFormatter

In [ ]:
# Funções para alterar a forma de exposição dos valores no gráfico
def currency_eixo(x, pos):
    """The two args are the value and tick position"""
    if x >= 1e6:
        s = 'R${:4.0f}'.format(x*1e-3) # {:<width>.<prec><type>}
    else:
        s = 'R${:4.0f}'.format(x*1e-3)
    return s
def porcentagem_eixo(x, pos):
    s = '{:4.0f}%'.format(x)
    return s
# ou aplicar diretamente: lambda x, pos: '{:4.0f}%'.format(x)
def formatar_reais(x, pos):
    s = 'R$ {:,.0f}'.format(x).replace(',', '.')
    return s
def formatar_reais_e_centavos(x):
    s = 'R$ {:,.2f}'.format(x).replace(',', 'X').replace('.', ',').replace('X', '.')
    return s
def formatar_milhares(x, pos):
    s = '{:4.0f}'.format(x*1e-3)
    return s
def formatar_bilhoes(x, pos):
    s = '{:4.2f}B'.format(x*1e-9)
    return s
def formatar_ano(x, pos):
    return '{:4.0f}'.format(x)
def format_thousands(x, pos):
    return '{:,.0f}'.format(x).replace(',', '.')

In [ ]:
print(plt.style.available)
plt.style.use('seaborn-v0_8') # ggplot também é bom
#plt.style.use('ggplot')
#plt.style.use('bmh')
#plt.style.use('seaborn')

In [ ]:
plt.rc('figure', figsize=(6,3), titlesize=15)
plt.rc('lines', linewidth=2)
plt.rc('axes', labelsize=8, titlesize=8, titleweight='bold', titley=1.05, labelpad=6) # vai mudar do x e do y
plt.rc('xtick', labelsize=8)
plt.rc('ytick', labelsize=8)
plt.rc('legend', loc='lower center', fontsize=8, frameon=True, framealpha=0.8)
colors = plt.colormaps.get_cmap('Set1') # bom para qualitativo
plt.colormaps.get_cmap('Set1') # melhor porque distingue melhor as primeiras cores

In [ ]:
tab_finalidade_ano.loc[tab_finalidade_ano['finalidade'] == 'Comercial', 'finalidade'] = 'Unidades Comerciais'
tab_finalidade_ano.loc[tab_finalidade_ano['finalidade'] == 'Residencial', 'finalidade'] = 'Unidades Residenciais'
tab_finalidade_ano.loc[tab_finalidade_ano['finalidade'] == 'Loteamento', 'finalidade'] = 'Lotes'

In [ ]:
fig, ax = plt.subplots()

finalidades = tab_finalidade_ano['finalidade'].unique()
colors = plt.colormaps.get_cmap('Set1')

#patterns = ['xxx', 'ooo', '...', '\\', '.', 'x']  # List of patterns to use
#for i, (finalidade, pattern) in enumerate(zip(finalidades, patterns)):
for i, finalidade in enumerate(finalidades):
    subset = tab_finalidade_ano[tab_finalidade_ano['finalidade'] == finalidade]
    ax.bar(subset['ano_entrega'] + i*0.25, # i*width
           subset['n_total_de_unidades'],
           width=0.25, label=finalidade, color=colors(i), alpha=0.7, #hatch=pattern
          )
# ticks
ax.xaxis.set_major_formatter(FuncFormatter(formatar_ano))
#ax.set_xticks(tab_finalidade_ano['ano_entrega'].dropna().unique() + 0.2 * (len(finalidades) - 1) / 2)
ax.set_xticks(tab_finalidade_ano['ano_entrega'].unique())
ax.xaxis.set_tick_params(rotation=45)
ax.yaxis.set_major_formatter(FuncFormatter(format_thousands))
# #labels
#ax.set_title('Número de Unidades Lançadas por Finalidade', loc='center')
# #legend
leg = ax.legend(bbox_to_anchor=(0.5, -0.28),
                ncol=len(finalidades))
leg.get_frame().set_edgecolor('black')
leg.get_frame().set_linewidth(0.4)
ax.set_ylabel('Total de Unidades Lançadas') #ax.set_xlabel('Ano de Entrega')

plt.show()

In [ ]:
fig.savefig(fr"{path_imagens}\finalidade_por_ano(matplotlib).png",
            dpi=200,  bbox_inches="tight")

# Mercado Residencial


## Filtro

In [ ]:
df = df.query("finalidade == 'Residencial'")

In [ ]:
len(df)

In [ ]:
df = df[df['n_total_de_unidades'].notnull()]

In [ ]:
df = df[df['n_total_de_unidades']!=0]

In [ ]:
len(df)

In [ ]:
df['rgi'].nunique()

## Explodir Dataframe

In [ ]:
repeated_idx = df.index.repeat(df['n_total_de_unidades'])
df_explode = df.loc[repeated_idx].reset_index(drop=True)

In [ ]:
len(df_explode)

In [ ]:
df_explode.info()

In [ ]:
df['rgi'].nunique()

## Tipos

In [ ]:
tab_unidades = df.pivot_table(index= 'unidade',
                              values = 'n_total_de_unidades',
                              aggfunc = 'sum')
tab_unidades.reset_index(inplace=True)
tab_unidades['percent'] = (
        tab_unidades['n_total_de_unidades'] / tab_unidades['n_total_de_unidades'].sum())
tab_unidades.sort_values(by='percent', inplace=True, ascending=False)
tab_unidades.head(10)

In [ ]:
tab_unidades.to_excel(fr'{path_imagens}\tipos_de_unidades.xlsx', index=False)

## Padrão
Mostrar a distribuição dos tipos ao longo do tempo

In [ ]:
df['classificacao_do_tipo'].value_counts()

In [ ]:
tab_unidades = df.pivot_table(index= ['ano_entrega', 'classificacao_do_tipo'],
                              values = 'n_total_de_unidades',
                              aggfunc = 'sum')
tab_unidades.reset_index(inplace=True)
tab_unidades['perct'] = tab_unidades.groupby(
    'ano_entrega')['n_total_de_unidades'].apply(lambda x: 100 * x / x.sum()).values

tab_unidades.head(5)

In [ ]:
category_orders = {'classificacao_do_tipo':
                         ['ECONÔMICO', 'MÉDIO BAIXO', 'MÉDIO', 'MÉDIO ALTO', 'ALTO']}

In [ ]:
fig = px.bar(tab_unidades,
               x = 'ano_entrega',
               y = 'n_total_de_unidades',
               color = 'classificacao_do_tipo',
              barmode = 'stack',
              category_orders = category_orders)
fig

In [ ]:
#fig.write_image(fr"{path_imagens}\padrao_por_ano_ABSOLUTO.png")

Matplotlib

In [ ]:
tab_unidades['classificacao_do_tipo'] = pd.Categorical(tab_unidades['classificacao_do_tipo'],
                                                       categories=category_orders['classificacao_do_tipo'],
                                                       ordered=True)
tab_unidades = tab_unidades.sort_values(by=['ano_entrega', 'classificacao_do_tipo'])

In [ ]:
fig, ax = plt.subplots()

years = tab_unidades['ano_entrega'].unique()
categories = tab_unidades['classificacao_do_tipo'].unique()

bottom = pd.Series([0]*len(years), index=years)
colors = plt.colormaps.get_cmap('Set1')
for i, cat in enumerate(categories):
    data = tab_unidades[tab_unidades['classificacao_do_tipo'] == cat]
    ax.bar(data['ano_entrega'],
           data['n_total_de_unidades'],
           bottom=bottom[data['ano_entrega']],
           label=cat,
           color=colors(i),
           alpha=0.7)
    bottom[data['ano_entrega']] += data['n_total_de_unidades'].values
# ticks
ax.xaxis.set_major_formatter(FuncFormatter(formatar_ano))
ax.set_xticks(tab_unidades['ano_entrega'].unique())
ax.xaxis.set_tick_params(rotation=45)
ax.yaxis.set_major_formatter(FuncFormatter(format_thousands))
# #labels
#ax.set_title('Número de Unidades Lançadas por Segmento Econômico', loc='center')
# #legend
leg = ax.legend(bbox_to_anchor=(0.5, -0.35),
                ncol=len(categories))
leg.get_frame().set_edgecolor('black')
leg.get_frame().set_linewidth(0.4)

ax.set_ylabel('Total de Unidades Lançadas') #ax.set_xlabel('Ano de Entrega')
plt.show()

In [ ]:
fig.savefig(fr"{path_imagens}\padrao_por_ano_ABSOLUTO(matplotlib).png",
            dpi=200,  bbox_inches="tight")

In [ ]:
fig = px.bar(tab_unidades,
               x = 'ano_entrega',
               y = 'perct',
               color = 'classificacao_do_tipo',
              barmode = 'stack',
              category_orders = category_orders)
fig

In [ ]:
fig.write_image(fr"{path_imagens}\padrao_por_ano_RELATIVO.png")

## Quantidades

In [ ]:
tab_qtdd_unidades = df.pivot_table(index=['ano_entrega', 'cidade'],
                                   values= 'n_total_de_unidades',
                                   aggfunc='sum')
tab_qtdd_unidades.reset_index(inplace=True)
tab_qtdd_unidades['perct'] = tab_qtdd_unidades.groupby(
    'ano_entrega')['n_total_de_unidades'].apply(lambda x: 100 * x / x.sum()).values

tab_qtdd_unidades.head(5)

Matplotlib

In [ ]:
category_orders = {'cidade': ['BELO HORIZONTE', 'CONTAGEM', 'BETIM', 'NOVA LIMA', 'LAGOA SANTA']}
tab_qtdd_unidades['classificacao_do_tipo'] = pd.Categorical(tab_qtdd_unidades['cidade'],
                                                       categories=category_orders['cidade'],
                                                       ordered=True)
tab_qtdd_unidades = tab_qtdd_unidades.sort_values(by=['ano_entrega', 'cidade'])

In [ ]:
fig, ax = plt.subplots()

years = tab_qtdd_unidades['ano_entrega'].unique()
categories = category_orders['cidade']

bottom = pd.Series([0]*len(years), index=years)
colors = plt.colormaps.get_cmap('Set1')
for i, cat in enumerate(categories):
    data = tab_qtdd_unidades[tab_qtdd_unidades['cidade'] == cat]
    ax.bar(data['ano_entrega'],
           data['n_total_de_unidades'],
           bottom=bottom[data['ano_entrega']],
           label=cat,
           color=colors(i),
           alpha=0.7)
    bottom[data['ano_entrega']] += data['n_total_de_unidades'].values
# ticks
ax.xaxis.set_major_formatter(FuncFormatter(formatar_ano))
ax.set_xticks(tab_qtdd_unidades['ano_entrega'].unique())
ax.xaxis.set_tick_params(rotation=45)
ax.yaxis.set_major_formatter(FuncFormatter(format_thousands))
# #labels
#ax.set_title('Número de Unidades Lançadas por Cidade', loc='center')
# #legend
leg = ax.legend(bbox_to_anchor=(0.5, -0.35),
                ncol=len(finalidades))
leg.get_frame().set_edgecolor('black')
leg.get_frame().set_linewidth(0.4)

ax.set_ylabel('Total de Unidades Lançadas') #ax.set_xlabel('Ano de Entrega')
plt.show()

In [ ]:
fig.savefig(fr"{path_imagens}\padrao_por_cidade_ABSOLUTO(matplotlib).png",
            dpi=200,  bbox_inches="tight")

Plotly

In [ ]:
fig = px.bar(tab_qtdd_unidades,
               x = 'ano_entrega',
               y = 'n_total_de_unidades',
               color = 'cidade',
              barmode = 'stack',
              category_orders = category_orders)
fig

In [ ]:
fig.write_image(fr"{path_imagens}\qtdd_unidades_por_ano_e_cidade.png")

In [ ]:
df['n_total_de_unidades'].sum()

## Preço

In [ ]:
len(df)

In [ ]:
len(df_explode)

In [ ]:
df_explode['vuv_preco_m2_privativo_no_lancamento_def'] = (
    df_explode['vmu_preco_de_venda_da_unidade_no_lancamento_def'] /
    df_explode['area_privativa'])

In [ ]:
df_explode.head(5)

In [ ]:
df_explode['area_privativa'].describe()

In [ ]:
media = round(df_explode['vuv_preco_m2_privativo_no_lancamento_def'].mean(), 2)
mediana = round(df_explode['vuv_preco_m2_privativo_no_lancamento_def'].median(), 2)


fig = px.histogram(df_explode,
             x = ['vuv_preco_m2_privativo_no_lancamento_def'],
             nbins=100)
fig.add_vline(x = media,
              line=dict(color="red", width=3, dash="dashdot"))
fig.add_vline(x=mediana,
              line=dict(color="green", width=3, dash="dashdot"))

fig.add_annotation(#y=fig.data[0].y.max(),
    x=media, text=f"media = {media}", showarrow=True, arrowhead=1, ax=20, ay=-80)
fig.add_annotation(#y=fig.data[0].y.max(),
    x=mediana, text=f"mediana = {mediana}", showarrow=True, arrowhead=1, ax=-20, ay=-50)


fig.update_yaxes(title_text='Nº de Unidades')
fig.update_xaxes(title_text='Preço m² - Área Privativa')

In [ ]:
fig.write_image(fr"{path_imagens}\histograma_preco_m2.png")

Matplotlib

In [ ]:
colors

In [ ]:
alpha = 0.7

media = round(df_explode['vuv_preco_m2_privativo_no_lancamento_def'].mean(), 2)
mediana = round(df_explode['vuv_preco_m2_privativo_no_lancamento_def'].median(), 2)


fig, ax = plt.subplots()


ax.hist(df_explode['vuv_preco_m2_privativo_no_lancamento_def'],
        bins=70,
        alpha=alpha,
        edgecolor='black',
        color = colors(1))


ax.axvline(media, color='red', linestyle='dashdot', linewidth=1.5, alpha=alpha)
ax.axvline(mediana, color='green', linestyle='dashdot', linewidth=1.5,alpha=alpha)

ax.annotate(f'média = {formatar_reais_e_centavos(media)}', xy=(media, ax.get_ylim()[1]/2), xytext=(20, -20),
            textcoords='offset points', arrowprops=dict(arrowstyle="->", lw=0.8),
           fontsize=9, color = colors(0))
ax.annotate(f'mediana = {formatar_reais_e_centavos(mediana)}', xy=(mediana, ax.get_ylim()[1]*0.9), xytext=(20, -50),
            textcoords='offset points', arrowprops=dict(arrowstyle="->", lw=0.8),
           fontsize=9, color = colors(2))


ax.set_xlabel('Preço m² real')
ax.set_ylabel('Total de Unidades Lançadas')
# ticks
ax.xaxis.set_major_formatter(FuncFormatter(formatar_reais))
ax.yaxis.set_major_formatter(FuncFormatter(format_thousands))

ax.tick_params(axis='x', direction='out',
               which='major', length=5, color='gray')
# #labels
#ax.set_title('Distribuição do preço deflacionado do m² \n ano-base: 2023', loc='center')

plt.show()

In [ ]:
fig.savefig(fr"{path_imagens}\histograma_preco_m2_def(matplotlib).png",
            dpi=200,  bbox_inches="tight")

Preço por cidade

In [ ]:
tab_preco = df_explode.pivot_table(index= ['ano_entrega', 'cidade'],
                                   values = 'vuv_preco_m2_privativo_no_lancamento_def',
                                   aggfunc = ['mean', 'median'])
tab_preco.reset_index(inplace=True)
tab_preco.head(3)

In [ ]:
tab_preco.columns = ['ano_entrega', 'cidade', 'MEDIA_preco_m2', 'MEDIANA_preco_m2']

In [ ]:
fig = px.line(tab_preco,
                x='ano_entrega',
                y = 'MEDIA_preco_m2',
                color = 'cidade')

fig.update_xaxes(title_text='Ano')
fig.update_yaxes(title_text='Média Preço m² - Área Privativa')

In [ ]:
fig.write_image(fr"{path_imagens}\media_preco_m2.png")

In [ ]:
fig = px.line(tab_preco,
                x='ano_entrega',
                y = 'MEDIANA_preco_m2',
                color = 'cidade')
fig.update_xaxes(title_text='Ano')
fig.update_yaxes(title_text='Mediana Preço m² - Área Privativa')

In [ ]:
fig.write_image(fr"{path_imagens}\mediana_preco_m2.png")

Matplotlib

In [ ]:
fig, ax = plt.subplots()

colors = plt.colormaps.get_cmap('Set1')

for i, city in enumerate(tab_preco['cidade'].unique()):
    city_data = tab_preco[tab_preco['cidade'] == city]
    ax.plot(city_data['ano_entrega'], city_data['MEDIA_preco_m2'],
            label=city, color = colors(i), alpha=alpha, marker = '.')

# ticks
ax.xaxis.set_major_formatter(FuncFormatter(formatar_ano))
ax.set_xticks(tab_preco['ano_entrega'].unique())
ax.xaxis.set_tick_params(rotation=45)
ax.yaxis.set_major_formatter(FuncFormatter(formatar_reais))
# #labels
#ax.set_title('Evolução da média do preço do m² deflacionado por cidade \n ano-base: 2023', loc='center')
# #legend
leg = ax.legend(bbox_to_anchor=(0.5, -0.35),
                ncol=3)
leg.get_frame().set_edgecolor('black')
leg.get_frame().set_linewidth(0.4)

ax.set_ylabel('Preço real do m²') #ax.set_xlabel('Ano de Entrega')

plt.show()

In [ ]:
fig.savefig(fr"{path_imagens}\evolucao_media_preco_m2(matplotlib).png",
            dpi=200,  bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots()

colors = plt.colormaps.get_cmap('Set1')

for i, city in enumerate(tab_preco['cidade'].unique()):
    city_data = tab_preco[tab_preco['cidade'] == city]
    ax.plot(city_data['ano_entrega'], city_data['MEDIANA_preco_m2'],
            label=city, color = colors(i), alpha=alpha, marker = '.')

# ticks
ax.xaxis.set_major_formatter(FuncFormatter(formatar_ano))
ax.set_xticks(tab_preco['ano_entrega'].unique())
ax.xaxis.set_tick_params(rotation=45)
ax.yaxis.set_major_formatter(FuncFormatter(formatar_reais))
# #labels
#ax.set_title('Evolução da mediana preço do m² deflacionado por cidade \n ano-base: 2023', loc='center')
# #legend
leg = ax.legend(bbox_to_anchor=(0.5, -0.35),
                ncol=3)
leg.get_frame().set_edgecolor('black')
leg.get_frame().set_linewidth(0.4)

ax.set_ylabel('Preço real do m²') #ax.set_xlabel('Ano de Entrega')

plt.show()

In [ ]:
fig.savefig(fr"{path_imagens}\evolucao_mediana_preco_m2(matplotlib).png",
            dpi=200,  bbox_inches="tight")

## VGV

In [ ]:
df['vgv_def'] = (df['vmu_preco_de_venda_da_unidade_no_lancamento_def']*df['n_total_de_unidades'])

In [ ]:
tab_vgv = df.pivot_table(index = ['ano_entrega', 'cidade'],
                       values = 'vgv_def',
                       aggfunc = 'sum')
tab_vgv.reset_index(inplace=True)
tab_vgv.head(3)

In [ ]:
category_orders = {'cidade': ['BELO HORIZONTE', 'CONTAGEM', 'BETIM', 'NOVA LIMA', 'LAGOA SANTA']}

In [ ]:
fig = px.bar(tab_vgv,
               x = 'ano_entrega',
               y = 'vgv_def',
               color = 'cidade',
              barmode = 'stack',
              category_orders = category_orders)
fig

In [ ]:
fig.write_image(fr"{path_imagens}\vgv_por_ano_e_cidade.png")

Matplotlib

In [ ]:
def formatar_bilhoes(x, pos):
    s = '{:.0f} B'.format(x*1e-9)
    return s

fig, ax = plt.subplots()

years = tab_vgv['ano_entrega'].unique()
categories = category_orders['cidade']

bottom = pd.Series([0]*len(years), index=years)
colors = plt.colormaps.get_cmap('Set1')
for i, cat in enumerate(categories):
    data = tab_vgv[tab_vgv['cidade'] == cat]
    ax.bar(data['ano_entrega'],
           data['vgv_def'],
           bottom=bottom[data['ano_entrega']],
           label=cat,
           color=colors(i),
           alpha=0.7)
    bottom[data['ano_entrega']] += data['vgv_def'].values
# ticks
ax.xaxis.set_major_formatter(FuncFormatter(formatar_ano))
ax.set_xticks(tab_qtdd_unidades['ano_entrega'].unique())
ax.xaxis.set_tick_params(rotation=45)
ax.yaxis.set_major_formatter(FuncFormatter(formatar_bilhoes))
# #labels
#ax.set_title('Valor Geral de Vendas (VGV) deflacionado por cidade \n ano-base: 2023', loc='center')
# #legend
leg = ax.legend(bbox_to_anchor=(0.5, -0.35),
                ncol=len(finalidades))
leg.get_frame().set_edgecolor('black')
leg.get_frame().set_linewidth(0.4)

ax.set_ylabel('VGV real (em bilhões R$)') #ax.set_xlabel('Ano de Entrega')
plt.show()

In [ ]:
fig.savefig(fr"{path_imagens}\evolucao_vgv_def(matplotlib).png",
            dpi=200,  bbox_inches="tight")

## Agentes

Incorporadores, construtores e vendedores

### Incorporação

#### unidades

In [ ]:
df['incorporadora_1'].value_counts(normalize=True) # no df, não no nº de unidades

In [ ]:
unid_por_incorp = df.pivot_table(index = 'incorporadora_1',
                                   values = 'n_total_de_unidades',
                                   aggfunc = 'sum')
unid_por_incorp.reset_index(inplace=True)
unid_por_incorp['participacao_no_total'] = (
    unid_por_incorp['n_total_de_unidades']/unid_por_incorp['n_total_de_unidades'].sum())
unid_por_incorp.sort_values(by='participacao_no_total', ascending=False, inplace=True)
unid_por_incorp['participacao_acumulada \n (concentração de mercado)'] = (
    unid_por_incorp['participacao_no_total'].cumsum())

In [ ]:
unid_por_incorp.head(10)

In [ ]:
unid_por_incorp.to_excel(fr'{path_imagens}\unidades_por_principais_incorporadoras.xlsx', index=False)

#### VGV

In [ ]:
vgv_por_incorp = df.pivot_table(index = 'incorporadora_1',
                                   values = 'vgv',
                                   aggfunc = 'sum')
vgv_por_incorp.reset_index(inplace=True)
vgv_por_incorp['participacao_no_total'] = (
    vgv_por_incorp['vgv']/vgv_por_incorp['vgv'].sum())
vgv_por_incorp.sort_values(by='participacao_no_total', ascending=False, inplace=True)
vgv_por_incorp['participacao_acumulada \n (concentração de mercado)'] = (
    vgv_por_incorp['participacao_no_total'].cumsum())

In [ ]:
vgv_por_incorp.head(10)

In [ ]:
vgv_por_incorp.to_excel(fr'{path_imagens}\vgv_por_principais_incorporadoras.xlsx', index=False)

## Construtora

#### unidades

In [ ]:
unid_por_const = df.pivot_table(index = 'construtora_1',
                                   values = 'n_total_de_unidades',
                                   aggfunc = 'sum')
unid_por_const.reset_index(inplace=True)
unid_por_const['participacao_no_total'] = (
    unid_por_const['n_total_de_unidades']/unid_por_const['n_total_de_unidades'].sum())
unid_por_const.sort_values(by='participacao_no_total', ascending=False, inplace=True)
unid_por_const['participacao_acumulada \n (concentração de mercado)'] = (
    unid_por_const['participacao_no_total'].cumsum())

In [ ]:
unid_por_const.head(10)

In [ ]:
unid_por_const.to_excel(fr'{path_imagens}\unidades_por_principais_construtoras.xlsx', index=False)

#### VGV

In [ ]:
vgv_por_const = df.pivot_table(index = 'construtora_1',
                                   values = 'vgv',
                                   aggfunc = 'sum')
vgv_por_const.reset_index(inplace=True)
vgv_por_const['participacao_no_total'] = (
    vgv_por_const['vgv']/vgv_por_const['vgv'].sum())
vgv_por_const.sort_values(by='participacao_no_total', ascending=False, inplace=True)
vgv_por_const['participacao_acumulada \n (concentração de mercado)'] = (
    vgv_por_const['participacao_no_total'].cumsum())

In [ ]:
vgv_por_const.head(10)

In [ ]:
vgv_por_const.to_excel(fr'{path_imagens}\vgv_por_principais_construtoras.xlsx', index=False)

## Vendedora

### unidades

In [ ]:
unid_por_vend = df.pivot_table(index = 'vendedora_1',
                                   values = 'n_total_de_unidades',
                                   aggfunc = 'sum')
unid_por_vend.reset_index(inplace=True)
unid_por_vend['participacao_no_total'] = (
    unid_por_vend['n_total_de_unidades']/unid_por_vend['n_total_de_unidades'].sum())
unid_por_vend.sort_values(by='participacao_no_total', ascending=False, inplace=True)
unid_por_vend['participacao_acumulada \n (concentração de mercado)'] = (
    unid_por_vend['participacao_no_total'].cumsum())

In [ ]:
unid_por_vend.head(10)

In [ ]:
unid_por_vend.to_excel(fr'{path_imagens}\unidades_por_principais_vendedoras.xlsx', index=False)

### VGV

In [ ]:
vgv_por_vend = df.pivot_table(index = 'vendedora_1',
                                   values = 'vgv',
                                   aggfunc = 'sum')
vgv_por_vend.reset_index(inplace=True)
vgv_por_vend['participacao_no_total'] = (
    vgv_por_vend['vgv']/vgv_por_vend['vgv'].sum())
vgv_por_vend.sort_values(by='participacao_no_total', ascending=False, inplace=True)
vgv_por_vend['participacao_acumulada \n (concentração de mercado)'] = (
    vgv_por_vend['participacao_no_total'].cumsum())

In [ ]:
vgv_por_vend.head(10)

In [ ]:
vgv_por_vend.to_excel(fr'{path_imagens}\vgv_por_principais_vendedoras.xlsx', index=False)

# Geodataframes

## Empreendimentos

queremos montar um dataframe em que observação base (linhas) é o empreendimento

In [ ]:
from pykml import parser
from shapely.geometry import Point, Polygon, LineString
import geopandas as gpd
import pandas as pd
from bs4 import BeautifulSoup


def parse_kml_to_gdf(kml_file):
    with open(kml_file, 'r') as f:
        root = parser.parse(f).getroot()

    features = []
    for placemark in root.Placemark:
        feature = {}


        feature['name'] = placemark.name.text if hasattr(placemark, 'name') else 'Unnamed'


        if hasattr(placemark, 'description'):
            description_html = placemark.description.text
            soup = BeautifulSoup(description_html, 'html.parser')
            tables = soup.find_all('table')

            if tables:
                for table in tables:
                    rows = table.find_all('tr')
                    for row in rows:
                        cols = row.find_all('td')
                        cols = [ele.text.strip() for ele in cols]
                        if len(cols) == 2:
                            key = cols[0].replace(' ', '_').replace(':', '').lower()
                            value = cols[1]
                            feature[key] = value


        if hasattr(placemark, 'Point'):
            coords = placemark.Point.coordinates.text.strip().split(',')
            geom = Point(float(coords[0]), float(coords[1]))
        elif hasattr(placemark, 'LineString'):
            coords = [tuple(map(float, coord.strip().split(','))) for coord in placemark.LineString.coordinates.text.strip().split()]
            geom = LineString(coords)
        elif hasattr(placemark, 'Polygon'):
            coords = [tuple(map(float, coord.strip().split(','))) for coord in placemark.Polygon.outerBoundaryIs.LinearRing.coordinates.text.strip().split()]
            geom = Polygon(coords)
        else:
            continue

        feature['geometry'] = geom
        features.append(feature)

    gdf = gpd.GeoDataFrame(features)
    return gdf

# Importar outras cidades também

In [ ]:
kml_file_bh = 'BH.kml'
gdf_bh = parse_kml_to_gdf(kml_file_bh)
gdf_bh['cidade'] = 'BELO HORIZONTE'

In [ ]:
kml_file_nova_lima = 'Nova Lima.kml'
gdf_nova_lima = parse_kml_to_gdf(kml_file_nova_lima)
gdf_nova_lima['cidade'] = 'NOVA LIMA'
kml_file_contagem = 'Contagem.kml'
gdf_contagem = parse_kml_to_gdf(kml_file_contagem)
gdf_contagem['cidade'] = 'CONTAGEM'
kml_file_betim = 'Betim.kml'
gdf_betim = parse_kml_to_gdf(kml_file_betim)
gdf_betim['cidade'] = 'BETIM'
kml_file_lagoa_santa = 'Lagoa Santa.kml'
gdf_lagoa_santa = parse_kml_to_gdf(kml_file_lagoa_santa)
gdf_lagoa_santa['cidade'] = 'LAGOA SANTA'

In [ ]:
gdf = pd.concat([gdf_bh, gdf_nova_lima, gdf_contagem, gdf_betim, gdf_lagoa_santa])

In [ ]:
gdf['cidade'].value_counts()

In [ ]:
len(df), df['codigo_no_mapa'].nunique(), len(gdf)

In [ ]:
# muitas obs irao ter coodernadas
len(df[df['empreendimento'].isin(gdf['name'].unique())])

In [ ]:
len(gdf), gdf['name'].nunique()
# isso vai dar problema no merge
# eu tenho mais de uma coordenada por empreendimento

In [ ]:
df_rgi = df.pivot_table(index='rgi',
                       values = ['n_total_de_unidades', 'vgv'],
                       aggfunc='sum')

df_rgi.reset_index(inplace=True)
df_rgi.columns = ['rgi', 'n_total_de_unidades_empreendimento', 'vgv']
len(df_rgi), df_rgi.head(2)

In [ ]:
colunas_de_interesse = ['rgi', 'data_lancamento', 'data_entrega', 'empreendimento',
                        'endereco', 'bairro', 'cidade',
                        'classificacao_do_tipo', 'incorporadora_1', 'construtora_1',
                        'vendedora_1', 'finalidade', 'tipologia', 'ano_entrega']

In [ ]:
df_rgi = df_rgi.merge(df.drop_duplicates('rgi')[colunas_de_interesse],
                     how = 'left',
                     on = 'rgi')

In [ ]:
df_rgi.head(2)

In [ ]:
len(df_rgi)

In [ ]:
len(gdf)

In [ ]:
len(gdf.drop_duplicates(subset=['name', 'lançamento', 'total_de_unidades']))

In [ ]:
for a, b in [('Jan/', '01-'), ('Fev/', '02-'), ('Mar/', '03-'), ('Abr/', '04-'),
             ('Mai/', '05-'), ('Jun/', '06-'), ('Jul/', '07-'), ('Ago/', '08-'),
             ('Set/', '09-'), ('Out/', '10-'), ('Nov/', '11-'), ('Dez/', '12-')]:
    gdf['lançamento'] = gdf['lançamento'].str.replace(a,b)

In [ ]:
gdf['lançamento'] = pd.to_datetime(gdf['lançamento'])

In [ ]:
df_rgi.head(1)

In [ ]:
df_rgi['identifier'] = (df_rgi['empreendimento'] +
                        df_rgi['n_total_de_unidades_empreendimento'].astype(int).astype(str) +
                        df_rgi['data_lancamento'].astype(str))
gdf['identifier'] = (gdf['name'] +
                    gdf['total_de_unidades'].astype(str) +
                    gdf['lançamento'].astype(str))
gdf.drop_duplicates(subset='identifier', inplace=True) # 4 drops
len(df_rgi), df_rgi['rgi'].nunique(), len(gdf), gdf['identifier'].nunique()

In [ ]:
df_rgi = df_rgi.merge(gdf[['identifier', 'geometry']],
                         how='left',
                         on = 'identifier')

ATENÇÃO: lembrar que muitos imóveis em df_rgi foram filtrados por isso nem todos os pontos em gdf serão transferidos para o df_rgi

In [ ]:
len(df_rgi)

In [ ]:
df_rgi = df_rgi[df_rgi['geometry'].notnull()]

In [ ]:
geometrias_validas = df_rgi.geometry.dropna()

df_rgi['x'] = geometrias_validas.apply(lambda geom: geom.x)
df_rgi['y'] = geometrias_validas.apply(lambda geom: geom.y)

In [ ]:
df_rgi.head(1)

In [ ]:
df_rgi.info()

In [ ]:
plt.scatter(df_rgi['x'],
            df_rgi['y'],
            s=df_rgi['n_total_de_unidades_empreendimento'],
            alpha=0.5,
            c=df_rgi['n_total_de_unidades_empreendimento'],
            cmap='viridis')

## Malha Municipios

In [ ]:
path_malha_municipios = r'D:\OneDrive\DADOS\outros dados\malhas\dados_ibge_br_municipios_2022'
#path_malha_municipios = r'C:\Users\Lord-\OneDrive\DADOS\outros dados\malhas\dados_ibge_br_municipios_2022'
malha_mun = gpd.read_file(fr"{path_malha_municipios}\BR_Municipios_2022.shp")
malha_mun.columns = malha_mun.columns.str.lower()
malha_mun = malha_mun[malha_mun['sigla_uf'] == 'MG']
rmbh = ['3106200', # bh
        '3105004', #baldim
        '3106705', #betim
        '3109006', #brumadinho
        '3110004', #caete
        '3112505', #capim branco
        '3117876', #confins
        '3118601', #contagem
        '3124104', #esmeralda
        '3126000', #florestal
        '3129806', #ibirite
        '3130101', #igarape
        '3132206', #itaguara
        '3133709', #itatiaiucu
        '3134608', #jaboticatubas
        '3136604', #nova uniao
        '3136652', #juatuba
        '3137601', #lagoa santa
        '3140159', #mario campos
        '3140704', #mateus leme
        '3141108', #matozinhos
        '3144805', #novalima
        '3149309', #pedro leopoldo
        '3153905', #raposos
        '3154606', #ribeirao das neves
        '3154804', #rio acima
        '3155306', #rio manso
        '3156700', #sabara
        '3157807', #santa luzia
        '3162922', #sao joaquim de bicas
        '3165537', #sarzedo
        '3168309', #taquaraçu
        '3171204', #vespasian#
        '3162955' # são jose da lapa
       ]
malha_dados = malha_mun[malha_mun['nm_mun'].isin(['Belo Horizonte', 'Lagoa Santa', 'Nova Lima',
                                                'Contagem', 'Betim'])]
malha_dados['nm_mun'] = malha_dados['nm_mun'].str.upper()
malha_mun['cd_mun'] = malha_mun['cd_mun'].astype(str)
malha_rmbh = malha_mun[malha_mun['cd_mun'].isin(rmbh)]

In [ ]:
malha_rmbh.plot()

## Malha regionais

In [ ]:
path_malha_regionais_bh = r'D:\OneDrive\DADOS\outros dados\malhas\malha_bh_regionais'
malha_regionais = gpd.read_file(fr"{path_malha_regionais_bh}\REGIONAL.shp")
malha_regionais = malha_regionais.to_crs(4674)
malha_regionais.columns = malha_regionais.columns.str.lower()

In [ ]:
malha_regionais.rename(columns={'nome':'nome_regional'}, inplace=True)

In [ ]:
malha_regionais.plot()

## Joining Layers

In [ ]:
bounding_box = malha_dados.geometry.bounds
xmin = bounding_box['minx'].min()
xmax = bounding_box['maxx'].max()
ymin = bounding_box['miny'].min()
ymax = bounding_box['maxy'].max()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

malha_rmbh.plot(ax=ax, color='blue', alpha = 0.1,
               edgecolor='black', linewidths=0.4)

malha_dados.plot(ax=ax, color='blue', alpha = 0.2,
               edgecolor='black', linewidths=0.8) # linewidths só funciona com edgecolor

malha_regionais.plot(ax=ax, color='blue', alpha = 0.2,
                     edgecolor='black', linewidths=1)

plt.scatter(df_rgi['x'],
            df_rgi['y'],
            s = 1, # size, df_rgi['n_total_de_unidades_empreendimento'],
            alpha=0.3,
            #c=df_rgi['n_total_de_unidades_empreendimento'], # cor
            #cmap='Reds' # escala de cores
           )
plt.xlim(xmin, xmax)
plt.ylim(ymin, ymax)

In [ ]:
fig.savefig(fr'{path_imagens}\empreendimentos.png',
           dpi=300, bbox_inches='tight', format='png')

# Analise georreferenciada

## Matplotlib Style (Maps)

In [ ]:
print(plt.style.available)

In [ ]:
plt.style.use('seaborn-v0_8')
plt.rc('figure', figsize=(5,10), titlesize=15)
plt.rc('lines', linewidth=2)
plt.rc('axes', labelsize=8, titlesize=8, titleweight='bold', titley=1.05, labelpad=6, grid=False) # vai mudar do x e do y
plt.rc('xtick', labelsize=8)
plt.rc('ytick', labelsize=8)
plt.rc('legend', loc='lower center', fontsize=8, frameon=True, framealpha=0.8)
colors = plt.colormaps.get_cmap('Set1') # bom para qualitativo
plt.colormaps.get_cmap('Set1') # melhor porque distingue melhor as primeiras cores

## Periodização

In [ ]:
conditions = [#(df_rgi['ano_entrega'] <= 2009),
              (df_rgi['ano_entrega'] > 2009) & (df_rgi['ano_entrega'] <= 2014),
              (df_rgi['ano_entrega'] > 2014) & (df_rgi['ano_entrega'] <= 2020),
              (df_rgi['ano_entrega'] > 2020)]

values = [#'-2009', # nao tem para esse filtro
          '2010-2014', '2015-2020', '2020-2023']
df_rgi['periodo'] = np.select(conditions,values)

In [ ]:
df_rgi['periodo'].value_counts()

In [ ]:
malha_rmbh.head(1)

In [ ]:
# for scale bar we need to calculate the pixel/distance relation
# https://geopandas.org/en/stable/gallery/matplotlib_scalebar.html
from shapely.geometry.point import Point
points = gpd.GeoSeries(
    [Point(-73.5, 40.5), Point(-74.5, 40.5)], crs=4326
)  # Geographic WGS 84 - degrees
points = points.to_crs(32619)  # Projected WGS 84 - meters
distance_meters = points[0].distance(points[1])

In [ ]:
distance_meters

In [ ]:
fig, ax = plt.subplots()
malha_rmbh.plot(ax=ax, color='blue', alpha = 0.1, edgecolor='black', linewidths=0.4)
malha_dados.plot(ax=ax, color='blue', alpha = 0.2, edgecolor='black', linewidths=0.8) # linewidths só funciona com edgecolor
malha_regionais.plot(ax=ax, color='blue', alpha = 0.2, edgecolor='black', linewidths=1)

hue_order = ['2010-2014', '2015-2020', '2020-2023']
sns.scatterplot(data=df_rgi,
                x='x', y='y',
                hue='periodo', hue_order=hue_order,
                alpha=0.4,
                palette= {'2010-2014': 'blue',
                          '2015-2020': 'red',
                          '2020-2023': 'yellow'},
                size = 1,
                sizes = (5, 100))
# alterar localização dos nomes dos municipios
for idx, row in malha_rmbh.iterrows():
    centroid = row['geometry'].centroid
    centroid_x = centroid.x
    centroid_y = centroid.y
    nome_municipio = row['nm_mun']
    if nome_municipio == 'Sabará':
        centroid_x = centroid_x*1.001
        centroid_y = centroid_y*1.002
    if nome_municipio == 'Esmeraldas':
        centroid_x = centroid_x*0.999
    if nome_municipio == 'São Joaquim de Bicas':
        nome_municipio = 'São Joaquim \n de Bicas'
    if nome_municipio == 'Raposos':
        centroid_x = centroid_x*1.0005
        centroid_y = centroid_y*0.9998
    # add o nome
    if xmin <= centroid_x <= xmax and ymin <= centroid_y <= ymax:
        ax.text(centroid_x, centroid_y, nome_municipio, fontsize=6, ha='center', color='black')

#ax.set_title('Localização dos empreendimentos \n de acordo com a periodização', loc='center')
# #legend
handles, labels = ax.get_legend_handles_labels()
leg = ax.legend(handles[0:-1], labels[0:-1],
                loc = 'center',
                bbox_to_anchor=(1.2, 0.5),
                title = 'Periodização',
                ncol=1)
leg.get_frame().set_edgecolor('black')
leg.get_frame().set_linewidth(0.4)

#scalebar = ScaleBar(distance_meters, #units="km", dimension="si-length",
#                    length_fraction=0.25, location="lower right", scale_loc="top", box_alpha=0.0)
#ax.add_artist(scalebar)

plt.xlim(xmin, xmax)
plt.ylim(ymin, ymax)
plt.show()

In [ ]:
labels

In [ ]:
fig.savefig(fr'{path_imagens}\2.empreendimentos_periodizacao.png',
            dpi=300, bbox_inches='tight', format='png')

In [ ]:
df_rgi.head(1)

## Nº Unidades

In [ ]:
fig, ax = plt.subplots(figsize=(5, 6))
malha_rmbh.plot(ax=ax, color='blue', alpha = 0.1,
               edgecolor='black', linewidths=0.4)
malha_dados.plot(ax=ax, color='blue', alpha = 0.2,
               edgecolor='black', linewidths=0.8) # linewidths só funciona com edgecolor
malha_regionais.plot(ax=ax, color='blue', alpha = 0.2,
                     edgecolor='black', linewidths=1)

sns.scatterplot(data=df_rgi,
                x='x', y='y',
                alpha=0.4,
                size = 'n_total_de_unidades_empreendimento',
                sizes = (1, 200))
ax.legend(loc='right', bbox_to_anchor=(-0.15, 0.5), title='Period')
plt.xlim(xmin, xmax)
plt.ylim(ymin, ymax)

In [ ]:
df_rgi.to_csv('empreendimentos.csv', index=False)

In [ ]:
df_rgi

In [ ]:
fig.savefig(fr'{path_imagens}\3.empreendimentos_volume_unidades.png',
            dpi=300, bbox_inches='tight', format='png')

## Agentes

In [ ]:
top_5_incorporadoras = df_rgi['incorporadora_1'].value_counts().head(5).index

In [ ]:
df_rgi.head(1)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 6))
malha_rmbh.plot(ax=ax, color='blue', alpha = 0.1,
               edgecolor='black', linewidths=0.4)
malha_dados.plot(ax=ax, color='blue', alpha = 0.2,
               edgecolor='black', linewidths=0.8) # linewidths só funciona com edgecolor
malha_regionais.plot(ax=ax, color='blue', alpha = 0.2,
                     edgecolor='black', linewidths=1)

hue_order = top_5_incorporadoras

sns.scatterplot(data=df_rgi[df_rgi['incorporadora_1'].isin(top_5_incorporadoras)],
                x='x', y='y',
                hue='incorporadora_1', hue_order=hue_order,
                alpha=0.4,
                #palette= {'2010-2014': 'blue','2015-2020': 'red','2020-2023': 'yellow'},
                size = 'n_total_de_unidades_empreendimento',
                sizes = (1, 200))
ax.legend(loc='right', bbox_to_anchor=(-0.15, 0.5), title='Period')
plt.xlim(xmin, xmax)
plt.ylim(ymin, ymax)

In [ ]:
fig.savefig(fr'{path_imagens}\3.empreendimentos_volume_e_construtora.png',
           dpi=300, bbox_inches='tight', format='png')

## Agregado Municipio e Regional BH

### Regionais BH

In [ ]:
df_rgi_bh = df_rgi[df_rgi['cidade'] == 'BELO HORIZONTE'].copy()
df_rgi_bh = gpd.GeoDataFrame(df_rgi_bh, geometry='geometry')
df_rgi_bh = df_rgi_bh.set_crs(4674)
len(df_rgi_bh)

In [ ]:
df_rgi_bh = gpd.sjoin(df_rgi_bh, malha_regionais[['nome_regional', 'geometry']],
                       how='left',
                       predicate='within')

In [ ]:
len(df_rgi_bh[df_rgi_bh['nome_regional'].isnull()])

In [ ]:
df_rgi_bh = df_rgi_bh[df_rgi_bh['nome_regional'].notnull()]

In [ ]:
tab_regional = df_rgi_bh.pivot_table(index='nome_regional',
                                      values = ['n_total_de_unidades_empreendimento',
                                                'vgv'],
                                      aggfunc = 'sum')
tab_regional.reset_index(inplace=True)

In [ ]:
malha_regionais = malha_regionais.merge(tab_regional, how='left', on = 'nome_regional')

### Outros Municipios

In [ ]:
df_rgi_not_bh = df_rgi[df_rgi['cidade'] != 'BELO HORIZONTE'].copy()
df_rgi_not_bh = gpd.GeoDataFrame(df_rgi_not_bh, geometry='geometry')
df_rgi_not_bh = df_rgi_not_bh.set_crs(4674)
len(df_rgi_not_bh)

In [ ]:
tab_municipios = df_rgi_not_bh.pivot_table(index='cidade',
                          values = ['n_total_de_unidades_empreendimento',
                                                'vgv'],
                                      aggfunc = 'sum')
tab_municipios.reset_index(inplace=True)

In [ ]:
tab_municipios

In [ ]:
malha_dados = malha_dados.merge(tab_municipios,
                                how='left',
                                left_on = 'nm_mun',
                                right_on = 'cidade')

In [ ]:
malha_dados_sem_bh = malha_dados[malha_dados['nm_mun']!='BELO HORIZONTE'].copy()

In [ ]:
malha_regionais.head(1)

### Agregating

In [ ]:
mun_e_regionais = pd.concat([malha_dados_sem_bh,malha_regionais])
mun_e_regionais = mun_e_regionais[['cidade', 'nome_regional',
                                 'geometry',
                                 'n_total_de_unidades_empreendimento',
                                 'vgv']]
mun_e_regionais.reset_index(drop=True, inplace=True)

In [ ]:
mun_e_regionais.head(2)

In [ ]:
mun_e_regionais['cidade_ou_regional'] = (
    mun_e_regionais['cidade'].combine_first(mun_e_regionais['nome_regional']))

In [ ]:
mun_e_regionais.drop(['cidade', 'nome_regional'], axis=1, inplace=True)

In [ ]:
mun_e_regionais.head(1)

In [ ]:
mun_e_regionais['y'] = mun_e_regionais['geometry'].centroid.y
mun_e_regionais['x'] = mun_e_regionais['geometry'].centroid.x

### Plot

In [ ]:
mun_e_regionais.head(1)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 6))
malha_rmbh.plot(ax=ax, color='blue', alpha = 0.1,
               edgecolor='black', linewidths=0.4)
malha_dados.plot(ax=ax, color='blue', alpha = 0.2,
               edgecolor='black', linewidths=0.8) # linewidths só funciona com edgecolor
malha_regionais.plot(ax=ax, color='blue', alpha = 0.2,
                     edgecolor='black', linewidths=1)
sns.scatterplot(data=mun_e_regionais,
                x='x', y='y',
                #hue='incorporadora_1', hue_order=hue_order,
                alpha=0.4,
                #palette= {'2010-2014': 'blue','2015-2020': 'red','2020-2023': 'yellow'},
                size = 'n_total_de_unidades_empreendimento',
                sizes = (10, 300))
ax.legend(loc='right', bbox_to_anchor=(-0.15, 0.5), title='Period')
plt.xlim(xmin, xmax)
plt.ylim(ymin, ymax)

In [ ]:
fig.savefig(fr'{path_imagens}\4.empreendimentos_agregado_regional_e_municipio.png',
           dpi=300, bbox_inches='tight', format='png')

# Duvidas

1) O que cada linha representa?
2) o que é rgi
3) Qual a relação do rgi com o código no Mapa e os empreendimentos?

In [ ]:
df_rgi

In [ ]:
df_rgi.to_excel(fr'empreendimentos.xlsx', index=False)

In [ ]:
print('sucesso')